# Exporting data from Earth Engine Cloud Storage

In [2]:
import numpy as np
import time
import ee
ee.Initialize()

### Recent SOC stocks
**Argentina SOC stock (0-30 cm)**

In [3]:
years = (np.arange(2)+1982).astype(np.str)#(np.arange(36)+1982).astype(np.str)
collection = 'users/iker/SOC_maps/SOC_stock_0_30'

Feb19_cstocks_1982_030_ll.tif
for year in years:
    start_date = f'{year}-01-01'
    end_date = f'{year}-12-31'
    
    collection = ee.ImageCollection(collection) 
        
    image = ee.Image(collection.filterDate(start_date, end_date).first())
    
    geometry = image.geometry()
    
    fileName = f'Feb19_cstocks_{str(year)}_030_ll'
    image_band = image.select(band)
        
    task = ee.batch.Export.image.toCloudStorage(
           image=image_band,
           bucket='vizz-data-transfer',
           fileNamePrefix=f'SOC_maps/SOC_concentration/{fileName}',
           description= fileName,
           scale= 250,
           region= geometry.getInfo().get('coordinates'),
           maxPixels=1e10)
    task.start()

**Argentina SOC concentration (0-5, 5-15, 30-60, 60-100, 100-200 cm)**

In [ ]:
years = (np.arange(36)+1982).astype(np.str)
quantiles = ['0.05', '0.5', '0.95']
collections = ['SOC_concentration_q5', 'SOC_concentration_q50', 'SOC_concentration_q95']
quantiles = dict(zip(quantiles, collections))

collection = ee.ImageCollection(f'users/iker/SOC_maps/{collections[1]}')
image = ee.Image(collection.first())
geometry = image.geometry()

bands = collection.first().bandNames().getInfo()
depth = ['2.5', '10', '22.5', '45', '80', '150']
bands = dict(zip(bands, depth))

for year in years:
    start_date = f'{year}-01-01'
    end_date = f'{year}-12-31'
    
    for quantile in quantiles.keys():
        
        collection = ee.ImageCollection(f'users/iker/SOC_maps/{quantiles[quantile]}') 
        
        image = ee.Image(collection.filterDate(start_date, end_date).first())
        
        for band in bands.keys():
            fileName = f'SOC_{str(year)}_q{quantile}_d{bands[band]}'
            image_band = image.select(band)
                
            task = ee.batch.Export.image.toCloudStorage(
                   image=image_band,
                   bucket='vizz-data-transfer',
                   fileNamePrefix=f'SOC_maps/SOC_concentration/{fileName}',
                   description= fileName,
                   scale= 250,
                   region= geometry.getInfo().get('coordinates'),
                   maxPixels=1e10)
            task.start()

### Historic SOC stocks

In [3]:
file_name = 'SOCS_0_200cm_year_NoLU_10km'
asset = f'users/iker/SOC_maps/{file_name}'

image = ee.Image(asset)

geometry = image.geometry()

task = ee.batch.Export.image.toCloudStorage(
       image=image,
       bucket='vizz-data-transfer',
       fileNamePrefix=f'SOC_maps/Historic/{file_name}',
       description= file_name,
       scale= 10000,
       region= geometry.getInfo().get('coordinates'),
       maxPixels=1e10,
      fileFormat= 'GeoTIFF',
      formatOptions= {'cloudOptimized': True})
task.start()

# Monitoring image export task
status = task.status().get('state')
while not task.status().get('state') == 'COMPLETED':
    print('Temporal status: ', task.status().get('state'))

    time.sleep(10)
    
print('Final status: ', task.status().get('state'))

Temporal status:  READY
Temporal status:  RUNNING
Temporal status:  RUNNING
Temporal status:  RUNNING
Temporal status:  RUNNING
Temporal status:  RUNNING
Temporal status:  RUNNING
Temporal status:  RUNNING
Temporal status:  RUNNING
Temporal status:  RUNNING
Final status:  COMPLETED
